# Snow Density Calculations - Kim & Jamieson (2014) Table 2 Method

This notebook demonstrates methods for calculating density from common snow pit measurements (hand hardness and grain form) using the "kim_table2" method. This method uses the Kim & Jamieson (2014) updated formulas based on Geldsetzer et al. (2000) for calculating density from hand hardness and grain form. The analysis uses the local snowpyt_mechparams package and snowpylot for CAAML parsing.


In [1]:
# Import Libraries
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Add the src directory to the path to import snowpyt_mechparams
sys.path.append('../src')
from snowpilot_utils import convert_grain_form, parse_sample_pits
from snowpyt_mechparams import density


Parse Snowpit Files

In [2]:
# Parse all snowpit files from the data folder
all_pits = parse_sample_pits('data')


Successfully parsed 50278 files
Failed to parse 0 files


Collect relevant data for each pit and convert to form needed to implement Kim_Geldsetzer

In [3]:
# Collect relevant data from each snowpit
pit_info = []
layer_info = []

for pit in all_pits:
    pit_dict = {
        'pit_id': pit.core_info.pit_id,
        'layer_count': len(pit.snow_profile.layers),
    }
    pit_info.append(pit_dict)

    for layer in pit.snow_profile.layers:
        # Create base layer dictionary
        layer_dict = {
            'pit_id': pit.core_info.pit_id,
            'hand_hardness': layer.hardness,
            'grain_form_primary': layer.grain_form_primary,
        }

        # Add geldsetzer grain form conversion if grain form data exists
        if layer.grain_form_primary:
            layer_dict['kim_jamieson_table2_grain_form'] = convert_grain_form(layer.grain_form_primary, 'kim_jamieson_table2')
        else:
            layer_dict['kim_jamieson_table2_grain_form'] = None

        layer_info.append(layer_dict)

# Create a dataframe from the pit and layer info
pit_df = pd.DataFrame(pit_info)
layer_df = pd.DataFrame(layer_info)


Calculate Density using Kim & Jamieson (2014) Table 2 Method

In [4]:
# Calculate density using Kim_Geldsetzer method for each layer
def calculate_layer_density(row):
    try:
        # Only calculate if we have a mapped grain form and hand hardness
        if pd.isna(row['kim_jamieson_table2_grain_form']) or pd.isna(row['hand_hardness']):
            return pd.Series([np.nan, np.nan])

        # The density function returns a ufloat object with nominal value and uncertainty
        density_ufloat = density.calculate_density(
            method='kim_jamieson_table2',
            hand_hardness=row['hand_hardness'],
            grain_form=row['kim_jamieson_table2_grain_form']
        )

        # Extract nominal value and standard deviation from ufloat object
        density_val = density_ufloat.nominal_value  # or density_ufloat.n
        density_unc = density_ufloat.std_dev        # or density_ufloat.s

        return pd.Series([density_val, density_unc])
    except Exception as e:
        print(f"Error calculating density for layer {row['pit_id']}: {e}")
        return pd.Series([np.nan, np.nan])

# Add density and uncertainty columns to the dataframe using apply method
# Calculate density for all rows - the function handles missing data internally
layer_df[['density', 'total_uncertainty']] = layer_df.apply(calculate_layer_density, axis=1)

# Calculate relative uncertainty
layer_df['relative_uncertainty'] = (layer_df['total_uncertainty'] / layer_df['density']) * 100


In [5]:
#layer_df.to_csv('geldsetzer_density_results.csv', index=False)


In [6]:
## Summary Stats

# Calculate and display summary statistics
print("=== Kim & Jamieson (2014) Table 2 APPLICATION SUMMARY ===")
print()

# Total Pits
total_pits = layer_df['pit_id'].nunique()
print(f"Total Pits with layers: {total_pits:,}")

# Total Layers
total_layers = len(layer_df)
print(f"Total Layers: {total_layers:,}")

# Total Layers with Hand Hardness and Grain Form Info
# Check for layers that have both hand hardness and grain form data
layers_with_data = layer_df[(layer_df['hand_hardness'].notna()) & 
                           (layer_df['grain_form_primary'].notna())]
total_layers_with_data = len(layers_with_data)
print(f"Total Layers with Hand Hardness and Grain Form Info: {total_layers_with_data:,}, "
      f"{total_layers_with_data/total_layers*100:.1f}%")

# Total Layers with Successfully Calculated Density
layers_with_density = layer_df[layer_df['density'].notna()]
total_successful_density = len(layers_with_density)
print(f"Total Layers with Successfully Calculated Density: {total_successful_density:,}, "
      f"{total_successful_density/total_layers*100:.1f}%")

# Average Relative Uncertainty
avg_relative_uncertainty = layer_df['relative_uncertainty'].mean()
print(f"Average Relative Uncertainty: {avg_relative_uncertainty:.2f}%")


=== Kim & Jamieson (2014) Table 2 APPLICATION SUMMARY ===

Total Pits with layers: 50,147
Total Layers: 371,429
Total Layers with Hand Hardness and Grain Form Info: 279,331, 75.2%
Total Layers with Successfully Calculated Density: 235,522, 63.4%
Average Relative Uncertainty: 18.07%
